Text-to-SQL Project: Data Ingestion & SQLite Setup

This notebook initializes the data pipeline for the **Text-to-SQL Eagles Assistant**, loading structured football data into a local SQLite database for natural language querying.

---

## 🧱 Notebook Purpose

Prepare and persist the core data tables used for Text-to-SQL translation:

- ✅ Load raw CSVs (player, game, season level) (Data from nflverse https://github.com/nflverse)
- ✅ Perform light cleaning and type conversions
- ✅ Save cleaned data to `stats.sqlite` database
- ✅ Verify schema and row counts for each table

---

## 📂 Tables Created

| Table Name      | Description                         |
|-----------------|-------------------------------------|
| `players`       | Season-level player stats           |
| `player_week`   | Game-level player stats             |
| `games`         | Game-level team stats               |
| `seasons`       | Season-level team summary stats     |

---

## 📁 Data Sources

- `stats_player_regpost_2020_2025.csv`
- `stats_player_week_regpost_2020_2025.csv`
- `eagles_game_summary_by_week_with_srs_cumleague_z_rank_2020_2025.csv`
- `eagles_season_summary_with_srs_cumleague_z_rank_2020_2025.csv`

All datasets are stored in the `/data/` directory and loaded dynamically via the `paths.py` module.

---

## 🛠️ Dependencies

- `pandas`
- `sqlite3`
- Custom `paths.py` for file management

---

## 🧠 Next Step

Once data is loaded and validated, proceed to `stats__texttosql_sqllite_load_02.ipynb` to build SQL prompts and generate queries using natural language questions.


In [ ]:
import pandas as pd
import numpy as np
import os

#set cwd to parent dir
os.chdir("..")

# -----------------------------
# Config
# -----------------------------
START_YEAR = 2020
END_YEAR   = 2025
TEAM = 'PHI'
# Optional Home Field Advantage to use in expected margin (0 to keep neutral; ~2.5 mimics PFR)
HOME_FIELD_ADV = 0.0


def process_season(year, team, home_field_adv=0.0):
    # -----------------------------
    # Load PBP
    # -----------------------------
    url = f"https://github.com/nflverse/nflverse-data/releases/download/pbp/play_by_play_{year}.csv.gz"
    pbp = pd.read_csv(url, compression='gzip', low_memory=False)

    # Regular + Postseason
    pbp = pbp[pbp['season_type'].isin(['REG', 'POST'])]

    # -----------------------------
    # Penalty-safe masks to align with official totals (PFR-style)
    # -----------------------------
    # Exclude accepted penalties / no-plays
    valid_play = (pbp['play_type'] != 'no_play')

    # Use attempt flags rather than play_type
    # - rush_attempt == 1 for rushing plays
    # - pass_attempt == 1 for passing plays (sacks counted in passing, not rushing)
    rush_mask = valid_play & (pbp['rush_attempt'] == 1)
    pass_mask = valid_play & (pbp['pass_attempt'] == 1)

    # -----------------------------
    # Base game table (Eagles)
    # -----------------------------
    game_scores = pbp[['game_id', 'week', 'home_team', 'away_team', 'home_score', 'away_score']].drop_duplicates()

    eagles_games = game_scores[(game_scores['home_team'] == team) | (game_scores['away_team'] == team)].copy()
    eagles_games['is_home'] = eagles_games['home_team'] == team
    eagles_games['opponent'] = np.where(eagles_games['is_home'], eagles_games['away_team'], eagles_games['home_team'])

    eagles_games['points_scored'] = np.where(eagles_games['is_home'], eagles_games['home_score'], eagles_games['away_score'])
    eagles_games['points_allowed'] = np.where(eagles_games['is_home'], eagles_games['away_score'], eagles_games['home_score'])
    eagles_games['margin_of_victory'] = eagles_games['points_scored'] - eagles_games['points_allowed']
    eagles_games['win'] = eagles_games['margin_of_victory'] > 0

    eagles_games = eagles_games.sort_values('week').reset_index(drop=True)
    eagles_games['cumulative_wins'] = eagles_games['win'].cumsum()
    eagles_games['cumulative_win_pct'] = eagles_games['cumulative_wins'] / (eagles_games.index + 1)

    # -----------------------------
    # Turnovers (committed/forced) - weekly
    # -----------------------------
    turnover_mask = (pbp['fumble_lost'] == 1) | (pbp['interception'] == 1)
    turnovers_committed = pbp[(pbp['posteam'] == team) & turnover_mask].groupby('game_id').size().reset_index(name='turnovers_committed')
    turnovers_forced   = pbp[(pbp['defteam'] == team) & turnover_mask].groupby('game_id').size().reset_index(name='turnovers_forced')

    turnovers = pd.merge(turnovers_committed, turnovers_forced, on='game_id', how='outer').fillna(0)
    turnovers['turnover_differential'] = turnovers['turnovers_forced'] - turnovers['turnovers_committed']

    eagles_results = pd.merge(eagles_games, turnovers, on='game_id', how='left')
    eagles_results[['turnovers_committed', 'turnovers_forced', 'turnover_differential']] = \
        eagles_results[['turnovers_committed', 'turnovers_forced', 'turnover_differential']].fillna(0).astype(int)

    eagles_results['cumulative_turnover_differential'] = eagles_results['turnover_differential'].cumsum()

    # -----------------------------
    # Helper to summarize play type (off & def EPA, yards) - weekly (penalty-safe)
    # -----------------------------
    def summarize_play_type(pbp_df, tm, play_type):
        if play_type == 'run':
            off = pbp_df[rush_mask & (pbp_df['posteam'] == tm)]
            deff = pbp_df[rush_mask & (pbp_df['defteam'] == tm)]
        elif play_type == 'pass':
            off = pbp_df[pass_mask & (pbp_df['posteam'] == tm)]
            deff = pbp_df[pass_mask & (pbp_df['defteam'] == tm)]
        else:
            raise ValueError("play_type must be 'run' or 'pass'")

        offense_summary = off.groupby('game_id').agg(
            avg_off_epa=('epa', 'mean'),
            total_yards=('yards_gained', 'sum'),
            total_attempts=('play_id', 'count'),
            total_tds=('touchdown', 'sum')
        ).reset_index()

        defense_summary = deff.groupby('game_id').agg(
            avg_def_epa=('epa', 'mean')
        ).reset_index()

        return pd.merge(offense_summary, defense_summary, on='game_id', how='outer').fillna(0)

    # Rush/pass summaries (offense + def EPA)
    run_summary = summarize_play_type(pbp, team, 'run').rename(columns={
        'avg_off_epa': 'rush_off_epa', 'avg_def_epa': 'rush_def_epa',
        'total_yards': 'rush_yards', 'total_attempts': 'rush_attempts', 'total_tds': 'rush_tds'
    })
    pass_summary = summarize_play_type(pbp, team, 'pass').rename(columns={
        'avg_off_epa': 'pass_off_epa', 'avg_def_epa': 'pass_def_epa',
        'total_yards': 'pass_yards', 'total_attempts': 'pass_attempts', 'total_tds': 'pass_tds'
    })
    play_summary = pd.merge(run_summary, pass_summary, on='game_id', how='outer').fillna(0)
    play_summary['total_tds'] = play_summary['rush_tds'] + play_summary['pass_tds']
    eagles_results = pd.merge(eagles_results, play_summary, on='game_id', how='left')

    # -----------------------------
    # Total Off/Def EPA and Yards (weekly, penalty-safe)
    # -----------------------------
    off_valid = pbp[(pbp['posteam'] == team) & (rush_mask | pass_mask)]
    def_valid = pbp[(pbp['defteam'] == team) & (rush_mask | pass_mask)]

    offense_epa = off_valid.groupby('game_id').agg(total_off_epa=('epa', 'sum')).reset_index()
    defense_epa = def_valid.groupby('game_id').agg(total_def_epa=('epa', 'sum')).reset_index()
    eagles_results = pd.merge(eagles_results, pd.merge(offense_epa, defense_epa, on='game_id', how='outer'),
                              on='game_id', how='left')

    offense_yards = off_valid.groupby('game_id').agg(total_yards_gained=('yards_gained', 'sum')).reset_index()
    defense_yards = def_valid.groupby('game_id').agg(total_yards_allowed=('yards_gained', 'sum')).reset_index()
    yards_summary = pd.merge(offense_yards, defense_yards, on='game_id', how='outer').fillna(0)
    eagles_results = pd.merge(eagles_results, yards_summary, on='game_id', how='left')

    # --- Run/Pass yards allowed (weekly, penalty-safe) ---
    rush_allowed = (pbp[rush_mask & (pbp['defteam'] == team)]
                    .groupby('game_id')['yards_gained'].sum()
                    .reset_index().rename(columns={'yards_gained':'rush_yards_allowed'}))
    pass_allowed = (pbp[pass_mask & (pbp['defteam'] == team)]
                    .groupby('game_id')['yards_gained'].sum()
                    .reset_index().rename(columns={'yards_gained':'pass_yards_allowed'}))
    defense_run_pass = rush_allowed.merge(pass_allowed, on='game_id', how='outer').fillna(0)
    eagles_results = pd.merge(eagles_results, defense_run_pass, on='game_id', how='left').fillna(0)

    # -----------------------------
    # Red Zone metrics (weekly offense & allowed)
    # (kept inclusive of penalty-driven entries; adjust with valid_play if desired)
    # -----------------------------
    def compute_rz_offense(df, tm):
        off = df[df['posteam'] == tm].copy()
        off['in_rz'] = off['yardline_100'] <= 20
        off['off_td_play'] = ((off.get('rush_touchdown', 0) == 1) | (off.get('pass_touchdown', 0) == 1))
        by_drive = (off.groupby(['game_id', 'drive'])
                      .agg(reached_rz=('in_rz', 'max'),
                           drive_td=('off_td_play', 'max'))
                      .reset_index())
        rz = by_drive[by_drive['reached_rz']]
        per_game = (rz.groupby('game_id')
                      .agg(rz_drives=('drive', 'count'),
                           rz_td_drives=('drive_td', 'sum'))
                      .reset_index())
        per_game['rz_td_pct'] = np.where(per_game['rz_drives'] > 0,
                                         per_game['rz_td_drives'] / per_game['rz_drives'], 0.0)
        return per_game

    def compute_rz_allowed(df, tm):
        opp = df[df['defteam'] == tm].copy()
        opp['in_rz'] = opp['yardline_100'] <= 20
        opp['off_td_play'] = ((opp.get('rush_touchdown', 0) == 1) | (opp.get('pass_touchdown', 0) == 1))
        by_drive = (opp.groupby(['game_id', 'drive'])
                      .agg(reached_rz=('in_rz', 'max'),
                           drive_td=('off_td_play', 'max'))
                      .reset_index())
        rz = by_drive[by_drive['reached_rz']]
        per_game = (rz.groupby('game_id')
                      .agg(rz_drives_allowed=('drive', 'count'),
                           rz_td_drives_allowed=('drive_td', 'sum'))
                      .reset_index())
        per_game['rz_td_pct_allowed'] = np.where(per_game['rz_drives_allowed'] > 0,
                                                 per_game['rz_td_drives_allowed'] / per_game['rz_drives_allowed'], 0.0)
        return per_game

    rz_off = compute_rz_offense(pbp, team)
    rz_def = compute_rz_allowed(pbp, team)
    eagles_results = pd.merge(eagles_results, rz_off, on='game_id', how='left')
    eagles_results = pd.merge(eagles_results, rz_def, on='game_id', how='left')

    for c in ['rz_drives','rz_td_drives','rz_drives_allowed','rz_td_drives_allowed']:
        if c in eagles_results:
            eagles_results[c] = eagles_results[c].fillna(0).astype(int)
    for c in ['rz_td_pct','rz_td_pct_allowed']:
        if c in eagles_results:
            eagles_results[c] = eagles_results[c].fillna(0.0)

    # -----------------------------
    # Weekly SRS (season-to-date up to each week)
    # -----------------------------
    def calculate_weekly_srs(pbp_data, max_week):
        games = pbp_data[pbp_data['week'] <= max_week]
        game_scores_w = games[['game_id', 'week', 'home_team', 'away_team', 'home_score', 'away_score']].drop_duplicates()

        teams = pd.unique(game_scores_w[['home_team', 'away_team']].values.ravel('K'))
        srs = {t: 0.0 for t in teams}
        mov = {t: [] for t in teams}
        opponents = {t: [] for t in teams}

        for _, row in game_scores_w.iterrows():
            home, away = row['home_team'], row['away_team']
            diff = row['home_score'] - row['away_score']
            mov[home].append(diff)
            mov[away].append(-diff)
            opponents[home].append(away)
            opponents[away].append(home)

        avg_mov = {t: (np.mean(mov[t]) if mov[t] else 0.0) for t in teams}

        for _ in range(100):
            new_srs = {}
            for t in teams:
                opp_srs = np.mean([srs[o] for o in opponents[t]]) if opponents[t] else 0.0
                new_srs[t] = avg_mov[t] + opp_srs
            if all(abs(new_srs[t] - srs[t]) < 1e-4 for t in teams):
                srs = new_srs
                break
            srs = new_srs

        mean_srs = np.mean(list(srs.values())) if len(srs) else 0.0
        srs = {t: rating - mean_srs for t, rating in srs.items()}
        return srs

    weeks = sorted(eagles_results['week'].unique())
    rows = []
    for w in weeks:
        srs_w = calculate_weekly_srs(pbp, w)
        for t, rating in srs_w.items():
            rows.append({'week': w, 'team': t, 'srs': rating})
    weekly_srs_df = pd.DataFrame(rows)

    phi_srs = weekly_srs_df[weekly_srs_df['team'] == team].rename(columns={'srs': 'phi_srs_week'})
    opp_srs = weekly_srs_df.rename(columns={'team': 'opponent', 'srs': 'opp_srs_week'})

    eagles_results = pd.merge(eagles_results, phi_srs[['week', 'phi_srs_week']], on='week', how='left')
    eagles_results = pd.merge(eagles_results, opp_srs[['week', 'opponent', 'opp_srs_week']], on=['week', 'opponent'], how='left')

    eagles_results['expected_mov_srs'] = (
        (eagles_results['phi_srs_week'] - eagles_results['opp_srs_week']) +
        np.where(eagles_results['is_home'], home_field_adv, -home_field_adv)
    )
    eagles_results['quality_of_win_srs'] = eagles_results['margin_of_victory'] - eagles_results['expected_mov_srs']

    # -----------------------------
    # Weekly sacks / QB hits / pressures (Eagles game-level, penalty-safe)
    # -----------------------------
    off_sacks_hits = (pbp[valid_play].groupby(['game_id','posteam'])
                        .agg(sacks_allowed=('sack','sum'),
                             qb_hits_allowed=('qb_hit','sum'))
                        .reset_index()
                        .rename(columns={'posteam':'team'}))

    def_sacks_hits = (pbp[valid_play].groupby(['game_id','defteam'])
                        .agg(sacks_caused=('sack','sum'),
                             qb_hits_caused=('qb_hit','sum'))
                        .reset_index()
                        .rename(columns={'defteam':'team'}))

    eagles_weekly_sh = (
        eagles_results[['game_id']].merge(off_sacks_hits[off_sacks_hits['team']==team][['game_id','sacks_allowed','qb_hits_allowed']],
                                          on='game_id', how='left')
                                  .merge(def_sacks_hits[def_sacks_hits['team']==team][['game_id','sacks_caused','qb_hits_caused']],
                                          on='game_id', how='left')
    )

    for c in ['sacks_allowed','qb_hits_allowed','sacks_caused','qb_hits_caused']:
        eagles_weekly_sh[c] = eagles_weekly_sh[c].fillna(0).astype(int)

    eagles_weekly_sh['pressures_allowed'] = eagles_weekly_sh['sacks_allowed'] + eagles_weekly_sh['qb_hits_allowed']
    eagles_weekly_sh['pressures_caused']  = eagles_weekly_sh['sacks_caused']  + eagles_weekly_sh['qb_hits_caused']

    eagles_results = eagles_results.merge(eagles_weekly_sh, on='game_id', how='left')

    # =============================================================================
    # League-wide cumulative (season-to-date) metrics, RAW sums, per-game, z-scores, and ranks
    # =============================================================================

    # 1) Team-game base: points & win points (1=win, 0.5=tie, 0=loss)
    games_all = pbp[['game_id','week','home_team','away_team','home_score','away_score']].drop_duplicates()
    diff = (games_all['home_score'] - games_all['away_score']).to_numpy()
    home_win_pts = (diff > 0).astype(float) + 0.5*(diff == 0)
    away_win_pts = (diff < 0).astype(float) + 0.5*(diff == 0)

    home_tbl = pd.DataFrame({
        'game_id': games_all['game_id'], 'week': games_all['week'],
        'team': games_all['home_team'],
        'points_for': games_all['home_score'], 'points_against': games_all['away_score'],
        'win_pts': home_win_pts
    })
    away_tbl = pd.DataFrame({
        'game_id': games_all['game_id'], 'week': games_all['week'],
        'team': games_all['away_team'],
        'points_for': games_all['away_score'], 'points_against': games_all['home_score'],
        'win_pts': away_win_pts
    })
    team_games = pd.concat([home_tbl, away_tbl], ignore_index=True)

    # 2) Per-team-game turnovers (committed/forced) and differential
    to_comm = (pbp[turnover_mask].groupby(['game_id','posteam']).size()
               .rename('turnovers_committed').reset_index()
               .rename(columns={'posteam':'team'}))
    to_forc = (pbp[turnover_mask].groupby(['game_id','defteam']).size()
               .rename('turnovers_forced').reset_index()
               .rename(columns={'defteam':'team'}))
    team_games = (team_games
                  .merge(to_comm, on=['game_id','team'], how='left')
                  .merge(to_forc, on=['game_id','team'], how='left'))
    team_games[['turnovers_committed','turnovers_forced']] = team_games[['turnovers_committed','turnovers_forced']].fillna(0).astype(int)
    team_games['tov_diff'] = team_games['turnovers_forced'] - team_games['turnovers_committed']

    # 3) Per-team-game yards & EPA (offense) - penalty-safe masks
    off_all = (pbp[(rush_mask | pass_mask)]
               .groupby(['game_id','posteam'])
               .agg(total_off_epa=('epa','sum'),
                    total_yards_gained=('yards_gained','sum'))
               .reset_index().rename(columns={'posteam':'team'}))

    off_run = (pbp[rush_mask]
               .groupby(['game_id','posteam'])
               .agg(rush_off_epa=('epa','sum'),
                    rush_yards=('yards_gained','sum'))
               .reset_index().rename(columns={'posteam':'team'}))

    off_pass = (pbp[pass_mask]
                .groupby(['game_id','posteam'])
                .agg(pass_off_epa=('epa','sum'),
                     pass_yards=('yards_gained','sum'))
                .reset_index().rename(columns={'posteam':'team'}))

    # 4) Per-team-game yards & EPA (defense/allowed) - penalty-safe masks
    def_all = (pbp[(rush_mask | pass_mask)]
               .groupby(['game_id','defteam'])
               .agg(total_def_epa=('epa','sum'),
                    total_yards_allowed=('yards_gained','sum'))
               .reset_index().rename(columns={'defteam':'team'}))

    def_run = (pbp[rush_mask]
              .groupby(['game_id','defteam'])
              .agg(rush_def_epa=('epa','sum'),
                   rush_yards_allowed=('yards_gained','sum'))
              .reset_index().rename(columns={'defteam':'team'}))

    def_pass = (pbp[pass_mask]
               .groupby(['game_id','defteam'])
               .agg(pass_def_epa=('epa','sum'),
                    pass_yards_allowed=('yards_gained','sum'))
               .reset_index().rename(columns={'defteam':'team'}))

    # Merge per-team-game stat blocks
    team_games = (team_games
        .merge(off_all, on=['game_id','team'], how='left')
        .merge(off_run, on=['game_id','team'], how='left')
        .merge(off_pass, on=['game_id','team'], how='left')
        .merge(def_all, on=['game_id','team'], how='left')
        .merge(def_run, on=['game_id','team'], how='left')
        .merge(def_pass, on=['game_id','team'], how='left')
    )

    for col in ['total_off_epa','total_yards_gained','rush_off_epa','rush_yards','pass_off_epa','pass_yards',
                'total_def_epa','total_yards_allowed','rush_def_epa','rush_yards_allowed','pass_def_epa','pass_yards_allowed']:
        team_games[col] = team_games[col].fillna(0.0)

    # 5) Red-zone per-team-game (offense + allowed) — kept inclusive of penalty-driven entries
    def rz_by_team_off(df):
        off = df.copy()
        off['in_rz'] = off['yardline_100'] <= 20
        off['off_td_play'] = ((off.get('rush_touchdown', 0) == 1) | (off.get('pass_touchdown', 0) == 1))
        g = (off.groupby(['game_id','posteam','drive'])
                .agg(reached_rz=('in_rz','max'),
                     drive_td=('off_td_play','max'))
                .reset_index())
        g = g[g['reached_rz']]
        per_game = (g.groupby(['game_id','posteam'])
                      .agg(rz_off_drives=('drive','count'),
                           rz_off_td_drives=('drive_td','sum'))
                      .reset_index().rename(columns={'posteam':'team'}))
        return per_game

    def rz_by_team_def(df):
        opp = df.copy()
        opp['in_rz'] = opp['yardline_100'] <= 20
        opp['off_td_play'] = ((opp.get('rush_touchdown', 0) == 1) | (opp.get('pass_touchdown', 0) == 1))
        g = (opp.groupby(['game_id','defteam','drive'])
                .agg(reached_rz=('in_rz','max'),
                     drive_td=('off_td_play','max'))
                .reset_index())
        g = g[g['reached_rz']]
        per_game = (g.groupby(['game_id','defteam'])
                      .agg(rz_def_drives=('drive','count'),
                           rz_def_td_drives=('drive_td','sum'))
                      .reset_index().rename(columns={'defteam':'team'}))
        return per_game

    rz_off_g = rz_by_team_off(pbp)
    rz_def_g = rz_by_team_def(pbp)
    team_games = (team_games
                  .merge(rz_off_g, on=['game_id','team'], how='left')
                  .merge(rz_def_g, on=['game_id','team'], how='left'))
    for c in ['rz_off_drives','rz_off_td_drives','rz_def_drives','rz_def_td_drives']:
        team_games[c] = team_games[c].fillna(0).astype(int)

    # 6) Sacks / QB hits / pressures per team-game (penalty-safe)
    tg_off_sh = (pbp[valid_play].groupby(['game_id','posteam'])
                   .agg(sacks_allowed=('sack','sum'),
                        qb_hits_allowed=('qb_hit','sum'))
                   .reset_index().rename(columns={'posteam':'team'}))
    tg_def_sh = (pbp[valid_play].groupby(['game_id','defteam'])
                   .agg(sacks_caused=('sack','sum'),
                        qb_hits_caused=('qb_hit','sum'))
                   .reset_index().rename(columns={'defteam':'team'}))
    team_games = (team_games
                  .merge(tg_off_sh, on=['game_id','team'], how='left')
                  .merge(tg_def_sh, on=['game_id','team'], how='left'))
    for c in ['sacks_allowed','qb_hits_allowed','sacks_caused','qb_hits_caused']:
        team_games[c] = team_games[c].fillna(0).astype(int)
    team_games['pressures_allowed'] = team_games['sacks_allowed'] + team_games['qb_hits_allowed']
    team_games['pressures_caused']  = team_games['sacks_caused']  + team_games['qb_hits_caused']

    # 7) Season-to-date cumulative stats per team by week (per-game AND RAW sums)
    team_games = team_games.sort_values(['team','week'])
    team_games['games_played'] = team_games.groupby('team').cumcount() + 1

    def csum(col): 
        return team_games.groupby('team')[col].cumsum()

    # Win %
    team_games['cum_win_pts'] = csum('win_pts')
    team_games['cum_win_pct'] = team_games['cum_win_pts'] / team_games['games_played']

    # Turnover differential (per-g and RAW sum)
    team_games['cum_tov_diff'] = csum('tov_diff')
    team_games['cum_tov_diff_per_g'] = team_games['cum_tov_diff'] / team_games['games_played']

    # Offense per-game cumulative + RAW sums
    off_pairs = [
        ('total_yards_gained','cum_yards_gained_per_g','cum_yards_gained_sum'),
        ('rush_yards','cum_rush_yards_per_g','cum_rush_yards_sum'),
        ('pass_yards','cum_pass_yards_per_g','cum_pass_yards_sum'),
        ('total_off_epa','cum_total_off_epa_per_g','cum_total_off_epa_sum'),
        ('rush_off_epa','cum_rush_off_epa_per_g','cum_rush_off_epa_sum'),
        ('pass_off_epa','cum_pass_off_epa_per_g','cum_pass_off_epa_sum'),
    ]
    for base, per_g, raw_sum in off_pairs:
        team_games[per_g]  = csum(base) / team_games['games_played']
        team_games[raw_sum] = csum(base)

    # Defense per-game cumulative + RAW sums
    def_pairs = [
        ('total_yards_allowed','cum_yards_allowed_per_g','cum_yards_allowed_sum'),
        ('rush_yards_allowed','cum_rush_yards_allowed_per_g','cum_rush_yards_allowed_sum'),
        ('pass_yards_allowed','cum_pass_yards_allowed_per_g','cum_pass_yards_allowed_sum'),
        ('total_def_epa','cum_total_def_epa_per_g','cum_total_def_epa_sum'),
        ('rush_def_epa','cum_rush_def_epa_per_g','cum_rush_def_epa_sum'),
        ('pass_def_epa','cum_pass_def_epa_per_g','cum_pass_def_epa_sum'),
    ]
    for base, per_g, raw_sum in def_pairs:
        team_games[per_g]  = csum(base) / team_games['games_played']
        team_games[raw_sum] = csum(base)

    # Cumulative red-zone efficiency (rates) + RAW sums
    team_games['cum_rz_off_drives'] = csum('rz_off_drives')
    team_games['cum_rz_off_td_drives'] = csum('rz_off_td_drives')
    team_games['cum_rz_def_drives'] = csum('rz_def_drives')
    team_games['cum_rz_def_td_drives'] = csum('rz_def_td_drives')
    team_games['cum_rz_off_td_pct'] = np.where(
        team_games['cum_rz_off_drives'] > 0,
        team_games['cum_rz_off_td_drives'] / team_games['cum_rz_off_drives'],
        0.0
    )
    team_games['cum_rz_def_td_pct_allowed'] = np.where(
        team_games['cum_rz_def_drives'] > 0,
        team_games['cum_rz_def_td_drives'] / team_games['cum_rz_def_drives'],
        0.0
    )

    # Sacks / QB hits / pressures cumulative per-g + RAW sums
    sh_pairs = [
        ('sacks_allowed','cum_sacks_allowed_per_g','cum_sacks_allowed_sum'),
        ('sacks_caused','cum_sacks_caused_per_g','cum_sacks_caused_sum'),
        ('qb_hits_allowed','cum_qb_hits_allowed_per_g','cum_qb_hits_allowed_sum'),
        ('qb_hits_caused','cum_qb_hits_caused_per_g','cum_qb_hits_caused_sum'),
        ('pressures_allowed','cum_pressures_allowed_per_g','cum_pressures_allowed_sum'),
        ('pressures_caused','cum_pressures_caused_per_g','cum_pressures_caused_sum'),
    ]
    for base, per_g, raw_sum in sh_pairs:
        team_games[per_g]  = csum(base) / team_games['games_played']
        team_games[raw_sum] = csum(base)

    # Optional points RAW sums
    team_games['cum_points_for_sum']     = csum('points_for')
    team_games['cum_points_against_sum'] = csum('points_against')
    team_games['cum_margin_sum']         = team_games['cum_points_for_sum'] - team_games['cum_points_against_sum']

    # 8) Per-week league z-scores for cumulative per-game metrics
    def zsafe(series):
        mu = series.mean()
        sd = series.std(ddof=0)
        if sd == 0 or np.isnan(sd):
            return (series - mu) * 0.0
        return (series - mu) / sd

    cum_cols = [
        'cum_win_pct',
        'cum_tov_diff_per_g',
        'cum_yards_gained_per_g','cum_yards_allowed_per_g',
        'cum_rush_yards_per_g','cum_pass_yards_per_g',
        'cum_rush_yards_allowed_per_g','cum_pass_yards_allowed_per_g',
        'cum_rz_off_td_pct','cum_rz_def_td_pct_allowed',
        'cum_total_off_epa_per_g','cum_rush_off_epa_per_g','cum_pass_off_epa_per_g',
        'cum_total_def_epa_per_g','cum_rush_def_epa_per_g','cum_pass_def_epa_per_g',
        'cum_sacks_allowed_per_g','cum_sacks_caused_per_g',
        'cum_qb_hits_allowed_per_g','cum_qb_hits_caused_per_g',
        'cum_pressures_allowed_per_g','cum_pressures_caused_per_g'
    ]
    for col in cum_cols:
        team_games[f'{col}_z_vs_league'] = team_games.groupby('week')[col].transform(zsafe)

    # 9) League ranks (season-to-date at each week)
    higher_is_better = [
        'cum_win_pct',
        'cum_tov_diff_per_g',
        'cum_yards_gained_per_g',
        'cum_rush_yards_per_g',
        'cum_pass_yards_per_g',
        'cum_rz_off_td_pct',
        'cum_total_off_epa_per_g','cum_rush_off_epa_per_g','cum_pass_off_epa_per_g',
        'cum_sacks_caused_per_g','cum_qb_hits_caused_per_g','cum_pressures_caused_per_g'
    ]
    lower_is_better = [
        'cum_yards_allowed_per_g',
        'cum_rush_yards_allowed_per_g',
        'cum_pass_yards_allowed_per_g',
        'cum_rz_def_td_pct_allowed',
        # defensive EPA: more negative is better → lower is better
        'cum_total_def_epa_per_g','cum_rush_def_epa_per_g','cum_pass_def_epa_per_g',
        'cum_sacks_allowed_per_g','cum_qb_hits_allowed_per_g','cum_pressures_allowed_per_g'
    ]
    for col in higher_is_better:
        team_games[f'{col}_rank_vs_league'] = team_games.groupby('week')[col].rank(method='min', ascending=False)
    for col in lower_is_better:
        team_games[f'{col}_rank_vs_league'] = team_games.groupby('week')[col].rank(method='min', ascending=True)

    # Include 'out of N' per week
    league_counts = team_games.groupby('week')['team'].nunique().rename('league_team_count').reset_index()
    team_games = team_games.merge(league_counts, on='week', how='left')

    # 10) Merge Eagles' RAW sums, PER-GAME metrics, ranks, and z-scores into eagles_results
    per_g_cols = [
        'cum_win_pct',
        'cum_tov_diff_per_g',
        'cum_yards_gained_per_g','cum_yards_allowed_per_g',
        'cum_rush_yards_per_g','cum_pass_yards_per_g',
        'cum_rush_yards_allowed_per_g','cum_pass_yards_allowed_per_g',
        'cum_total_off_epa_per_g','cum_rush_off_epa_per_g','cum_pass_off_epa_per_g',
        'cum_total_def_epa_per_g','cum_rush_def_epa_per_g','cum_pass_def_epa_per_g',
        'cum_sacks_allowed_per_g','cum_sacks_caused_per_g',
        'cum_qb_hits_allowed_per_g','cum_qb_hits_caused_per_g',
        'cum_pressures_allowed_per_g','cum_pressures_caused_per_g',
        'cum_rz_off_td_pct','cum_rz_def_td_pct_allowed'
    ]
    raw_sum_cols = [
        'cum_win_pts','cum_tov_diff',
        'cum_yards_gained_sum','cum_yards_allowed_sum',
        'cum_rush_yards_sum','cum_pass_yards_sum',
        'cum_rush_yards_allowed_sum','cum_pass_yards_allowed_sum',
        'cum_total_off_epa_sum','cum_rush_off_epa_sum','cum_pass_off_epa_sum',
        'cum_total_def_epa_sum','cum_rush_def_epa_sum','cum_pass_def_epa_sum',
        'cum_rz_off_drives','cum_rz_off_td_drives','cum_rz_def_drives','cum_rz_def_td_drives',
        'cum_sacks_allowed_sum','cum_sacks_caused_sum',
        'cum_qb_hits_allowed_sum','cum_qb_hits_caused_sum',
        'cum_pressures_allowed_sum','cum_pressures_caused_sum',
        'cum_points_for_sum','cum_points_against_sum','cum_margin_sum'
    ]
    z_cols    = [f'{c}_z_vs_league' for c in cum_cols]
    rank_cols = [f'{c}_rank_vs_league' for c in higher_is_better + lower_is_better]

    merge_cols = ['week','league_team_count'] + raw_sum_cols + per_g_cols + z_cols + rank_cols
    eagles_cum_all = team_games[team_games['team'] == team][merge_cols].copy()
    eagles_results = eagles_results.merge(eagles_cum_all, on='week', how='left')

    # add season column
    eagles_results['season'] = year

    # -----------------------------
    # Column Ordering: weekly → cumulative SUM → cumulative PER-GAME → cumulative RANK → cumulative Z
    # -----------------------------
    def add_if_exists(lst, cols):
        for c in cols:
            if c in eagles_results.columns:
                lst.append(c)

    ordered_cols = []
    # Identifiers / context
    add_if_exists(ordered_cols, ['season','game_id','week','is_home','opponent','league_team_count'])

    # Scoreline / record
    add_if_exists(ordered_cols, [
        'points_scored','points_allowed','margin_of_victory','win',
        'cum_points_for_sum','cum_points_against_sum','cum_margin_sum',
        'cumulative_wins','cumulative_win_pct',   # PHI-only
        'cum_win_pct','cum_win_pct_rank_vs_league','cum_win_pct_z_vs_league'
    ])

    # Turnovers
    add_if_exists(ordered_cols, [
        'turnovers_committed','turnovers_forced','turnover_differential',
        'cum_tov_diff','cum_tov_diff_per_g',
        'cum_tov_diff_per_g_rank_vs_league','cum_tov_diff_per_g_z_vs_league'
    ])

    # Total yards (off/def)
    add_if_exists(ordered_cols, [
        'total_yards_gained','total_yards_allowed',
        'cum_yards_gained_sum','cum_yards_allowed_sum',
        'cum_yards_gained_per_g','cum_yards_allowed_per_g',
        'cum_yards_gained_per_g_rank_vs_league','cum_yards_gained_per_g_z_vs_league',
        'cum_yards_allowed_per_g_rank_vs_league','cum_yards_allowed_per_g_z_vs_league'
    ])

    # Run yards (off/def)
    add_if_exists(ordered_cols, [
        'rush_yards','rush_yards_allowed',
        'cum_rush_yards_sum','cum_rush_yards_allowed_sum',
        'cum_rush_yards_per_g','cum_rush_yards_allowed_per_g',
        'cum_rush_yards_per_g_rank_vs_league','cum_rush_yards_per_g_z_vs_league',
        'cum_rush_yards_allowed_per_g_rank_vs_league','cum_rush_yards_allowed_per_g_z_vs_league'
    ])

    # Pass yards (off/def)
    add_if_exists(ordered_cols, [
        'pass_yards','pass_yards_allowed',
        'cum_pass_yards_sum','cum_pass_yards_allowed_sum',
        'cum_pass_yards_per_g','cum_pass_yards_allowed_per_g',
        'cum_pass_yards_per_g_rank_vs_league','cum_pass_yards_per_g_z_vs_league',
        'cum_pass_yards_allowed_per_g_rank_vs_league','cum_pass_yards_allowed_per_g_z_vs_league'
    ])

    # EPA total (off/def)
    add_if_exists(ordered_cols, [
        'total_off_epa','total_def_epa',
        'cum_total_off_epa_sum','cum_total_def_epa_sum',
        'cum_total_off_epa_per_g','cum_total_def_epa_per_g',
        'cum_total_off_epa_per_g_rank_vs_league','cum_total_off_epa_per_g_z_vs_league',
        'cum_total_def_epa_per_g_rank_vs_league','cum_total_def_epa_per_g_z_vs_league'
    ])

    # EPA rush (off/def)
    add_if_exists(ordered_cols, [
        'rush_off_epa','rush_def_epa',
        'cum_rush_off_epa_sum','cum_rush_def_epa_sum',
        'cum_rush_off_epa_per_g','cum_rush_def_epa_per_g',
        'cum_rush_off_epa_per_g_rank_vs_league','cum_rush_off_epa_per_g_z_vs_league',
        'cum_rush_def_epa_per_g_rank_vs_league','cum_rush_def_epa_per_g_z_vs_league'
    ])

    # EPA pass (off/def)
    add_if_exists(ordered_cols, [
        'pass_off_epa','pass_def_epa',
        'cum_pass_off_epa_sum','cum_pass_def_epa_sum',
        'cum_pass_off_epa_per_g','cum_pass_def_epa_per_g',
        'cum_pass_off_epa_per_g_rank_vs_league','cum_pass_off_epa_per_g_z_vs_league',
        'cum_pass_def_epa_per_g_rank_vs_league','cum_pass_def_epa_per_g_z_vs_league'
    ])

    # Red Zone (offense)
    add_if_exists(ordered_cols, [
        'rz_drives','rz_td_drives','rz_td_pct',
        'cum_rz_off_td_drives','cum_rz_off_drives',
        'cum_rz_off_td_pct','cum_rz_off_td_pct_rank_vs_league','cum_rz_off_td_pct_z_vs_league'
    ])

    # Red Zone (defense)
    add_if_exists(ordered_cols, [
        'rz_drives_allowed','rz_td_drives_allowed','rz_td_pct_allowed',
        'cum_rz_def_td_drives','cum_rz_def_drives',
        'cum_rz_def_td_pct_allowed','cum_rz_def_td_pct_allowed_rank_vs_league','cum_rz_def_td_pct_allowed_z_vs_league'
    ])

    # Sacks / Hits / Pressures (allowed)
    add_if_exists(ordered_cols, [
        'sacks_allowed','qb_hits_allowed','pressures_allowed',
        'cum_sacks_allowed_sum','cum_qb_hits_allowed_sum','cum_pressures_allowed_sum',
        'cum_sacks_allowed_per_g','cum_qb_hits_allowed_per_g','cum_pressures_allowed_per_g',
        'cum_sacks_allowed_per_g_rank_vs_league','cum_sacks_allowed_per_g_z_vs_league',
        'cum_qb_hits_allowed_per_g_rank_vs_league','cum_qb_hits_allowed_per_g_z_vs_league',
        'cum_pressures_allowed_per_g_rank_vs_league','cum_pressures_allowed_per_g_z_vs_league'
    ])

    # Sacks / Hits / Pressures (caused)
    add_if_exists(ordered_cols, [
        'sacks_caused','qb_hits_caused','pressures_caused',
        'cum_sacks_caused_sum','cum_qb_hits_caused_sum','cum_pressures_caused_sum',
        'cum_sacks_caused_per_g','cum_qb_hits_caused_per_g','cum_pressures_caused_per_g',
        'cum_sacks_caused_per_g_rank_vs_league','cum_sacks_caused_per_g_z_vs_league',
        'cum_qb_hits_caused_per_g_rank_vs_league','cum_qb_hits_caused_per_g_z_vs_league',
        'cum_pressures_caused_per_g_rank_vs_league','cum_pressures_caused_per_g_z_vs_league'
    ])

    # Attempts / TDs (weekly counts)
    add_if_exists(ordered_cols, ['rush_attempts','pass_attempts','total_tds'])

    # SRS contextual
    add_if_exists(ordered_cols, ['phi_srs_week','opp_srs_week','expected_mov_srs','quality_of_win_srs'])

    final_cols = [c for c in ordered_cols if c in eagles_results.columns]
    remaining = [c for c in eagles_results.columns if c not in final_cols]
    final_cols += remaining

    return eagles_results[final_cols]


# -----------------------------
# Run for span of years and save
# -----------------------------
all_seasons = []
for yr in range(START_YEAR, END_YEAR + 1):
    print(f"Processing season {yr}...")
    try:
        season_df = process_season(yr, TEAM, HOME_FIELD_ADV)
        all_seasons.append(season_df)
    except Exception as e:
        print(f"  Skipping {yr} due to error: {e}")

if all_seasons:
    eagles_all = pd.concat(all_seasons, ignore_index=True)

    # 1) Per-game CSV (all seasons, 1 row per game)
    games_outfile = f'2eagles_game_summary_by_week_with_srs_cumleague_z_rank_{START_YEAR}_{END_YEAR}.csv'
    eagles_all.to_csv(games_outfile, index=False)
    print(f"Saved per-game file with {len(eagles_all)} rows to '{games_outfile}'")

    # 2) Season summary CSV (1 row per season, end-of-season cumulative metrics)
    # Take the last (max-week) row per season
    season_summary = (eagles_all
                      .sort_values(['season', 'week'])
                      .groupby('season')
                      .tail(1)
                      .reset_index(drop=True))

    # Keep only season-level fields: season, week, league_team_count, and all cumulative metrics
    base_keep = ['season', 'week', 'league_team_count']
    cum_keep = [c for c in season_summary.columns if c.startswith('cum_')]
    season_summary = season_summary[base_keep + cum_keep]

    seasons_outfile = f'2eagles_season_summary_with_srs_cumleague_z_rank_{START_YEAR}_{END_YEAR}.csv'
    season_summary.to_csv(seasons_outfile, index=False)
    print(f"Saved season summary file with {len(season_summary)} rows to '{seasons_outfile}'")
else:
    print("No seasons processed successfully.")


Columns: ['player_id', 'player_name', 'player_display_name', 'position', 'position_group', 'headshot_url', 'season', 'season_type', 'recent_team', 'games', 'completions', 'attempts', 'passing_yards', 'passing_tds', 'passing_interceptions', 'sacks_suffered', 'sack_yards_lost', 'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards', 'passing_yards_after_catch', 'passing_first_downs', 'passing_epa', 'passing_cpoe', 'passing_2pt_conversions', 'pacr', 'carries', 'rushing_yards', 'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost', 'rushing_first_downs', 'rushing_epa', 'rushing_2pt_conversions', 'receptions', 'targets', 'receiving_yards', 'receiving_tds', 'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_air_yards', 'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa', 'receiving_2pt_conversions', 'racr', 'target_share', 'air_yards_share', 'wopr', 'special_teams_tds', 'def_tackles_solo', 'def_tackles_with_assist', 'def_tackle_assists', 'def_tackles_for_loss'

Now we'll download player stats. 

In [3]:
import pandas as pd
import os
from paths import DATA_DIR, PLAYERS_CSV, PLAYER_WEEKS_CSV

# ============================================================
# Config
# ============================================================
YEARS = range(2020, 2026)

URL_TEMPLATE_SEASON = (
    "https://github.com/nflverse/nflverse-data/releases/download/"
    "stats_player/stats_player_regpost_{year}.csv.gz"
)

URL_TEMPLATE_WEEK = (
    "https://github.com/nflverse/nflverse-data/releases/download/"
    "stats_player/stats_player_week_{year}.csv.gz"
)

os.makedirs(DATA_DIR, exist_ok=True)

# ============================================================
# Helper
# ============================================================
def download_and_concat(url_template, years, label):
    frames = []

    for year in years:
        url = url_template.format(year=year)
        print(f"Loading {label} {year} from {url} ...")

        try:
            df = pd.read_csv(url, compression="gzip", low_memory=False)

            if "season" not in df.columns:
                df["season"] = year
            else:
                df["season"] = df["season"].fillna(year)

            frames.append(df)
            print(f"  -> Loaded {df.shape[0]:,} rows")

        except Exception as e:
            print(f"  !! Failed to load {label} {year}: {e}")

    if not frames:
        raise RuntimeError(f"No {label} data files were loaded.")

    combined = pd.concat(frames, ignore_index=True)
    print(f"\n{label} combined size: {combined.shape[0]:,} rows")
    return combined


# ============================================================
# Download Season-Level
# ============================================================
season_df = download_and_concat(URL_TEMPLATE_SEASON, YEARS, "SEASON")
season_df.to_csv(PLAYERS_CSV, index=False)
print(f"Saved SEASON CSV → {PLAYERS_CSV}")

# ============================================================
# Download Week-Level
# ============================================================
week_df = download_and_concat(URL_TEMPLATE_WEEK, YEARS, "WEEK")
week_df.to_csv(PLAYER_WEEKS_CSV, index=False)
print(f"Saved WEEK CSV → {PLAYER_WEEKS_CSV}")


Loading SEASON 2020 from https://github.com/nflverse/nflverse-data/releases/download/stats_player/stats_player_regpost_2020.csv.gz ...
  -> Loaded 1,995 rows
Loading SEASON 2021 from https://github.com/nflverse/nflverse-data/releases/download/stats_player/stats_player_regpost_2021.csv.gz ...
  -> Loaded 2,088 rows
Loading SEASON 2022 from https://github.com/nflverse/nflverse-data/releases/download/stats_player/stats_player_regpost_2022.csv.gz ...
  -> Loaded 2,011 rows
Loading SEASON 2023 from https://github.com/nflverse/nflverse-data/releases/download/stats_player/stats_player_regpost_2023.csv.gz ...
  -> Loaded 1,948 rows
Loading SEASON 2024 from https://github.com/nflverse/nflverse-data/releases/download/stats_player/stats_player_regpost_2024.csv.gz ...
  -> Loaded 2,002 rows
Loading SEASON 2025 from https://github.com/nflverse/nflverse-data/releases/download/stats_player/stats_player_regpost_2025.csv.gz ...
  -> Loaded 1,883 rows

SEASON combined size: 11,927 rows
Saved SEASON CSV 

Lastly, we'll grab narrative data from wikipedia.

In [ ]:
import wikipediaapi
import os

# -----------------------------
# Config
# -----------------------------
titles = [
    "2020 Philadelphia Eagles season",
    "2021 Philadelphia Eagles season",
    "2022 Philadelphia Eagles season",
    "2023 Philadelphia Eagles season",
    "2024 Philadelphia Eagles season",
    "Jalen Hurts",
    "Nick Sirianni",
    "Philadelphia Eagles",
    "Run-pass option",
    "RPO (American football)"
]

output_dir = "data/narratives/"
os.makedirs(output_dir, exist_ok=True)

# -----------------------------
# Wikipedia API with User-Agent
# -----------------------------
wiki = wikipediaapi.Wikipedia(
    language="en",
    user_agent="FlyGPTBot/1.0 (lee.skelton@example.com)"
)

for title in titles:
    page = wiki.page(title)
    if page.exists():
        filename = title.replace(" ", "_").replace("/", "_").lower() + ".txt"
        filepath = os.path.join(output_dir, filename)

        with open(filepath, "w", encoding="utf-8") as f:
            f.write(page.text)
        print(f"✅ Saved: {title}")
    else:
        print(f"❌ Page not found: {title}")

        # Sanitize fil